In [2]:
import nltk
import time
import re
import numpy as np
import pandas as pd

from google.colab import drive 
drive.mount('/content/gdrive')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option("max_colwidth", 100)

Mounted at /content/gdrive


In [26]:
def clean(df):
  for col in df:
    df[col] = df[col].replace("\n", ' ', regex=True)
    df[col] = df[col].replace('\?\!{2,}', '?', regex=True)
    df[col] = df[col].replace('\.{1,}\!', '!', regex=True)
    df[col] = df[col].replace('\. \.\.\.', '. ', regex=True)
    df[col] = df[col].replace(' \.\.\. ', '. ', regex=True)
    df[col] = df[col].replace('\.\.\. ', '. ', regex=True)
    df[col] = df[col].replace('\.\,', '. ', regex=True)
    df[col] = df[col].replace('\.\?', '?', regex=True)
    df[col] = df[col].replace('\?\,', '? ', regex=True)
    df[col] = df[col].replace('\. \.', '. ', regex=True)
    df[col] = df[col].replace('\!{2,}', '!', regex=True)
    df[col] = df[col].replace('\?{2,}', '?', regex=True)
    df[col] = df[col].replace('\.{2,}', '.', regex=True)
    df[col] = df[col].replace('\_{2,}', '_', regex=True)
    df[col] = df[col].replace(' -- ', ', ', regex=True)
    df[col] = df[col].replace("\[", '', regex=True)
    df[col] = df[col].replace("\]", '', regex=True)
    df[col] = df[col].replace("“", '', regex=True)
    df[col] = df[col].replace("”", '', regex=True)
    df[col] = df[col].replace('"', '\'', regex=True) 
    df[col] = df[col].replace("’", '\'', regex=True)
    df[col] = df[col].replace("�s", '\'s', regex=True)
    df[col] = df[col].replace("�m", '\'m', regex=True)
    df[col] = df[col].replace("�re", '\'re', regex=True)
    df[col] = df[col].replace("�ve", '\'ve', regex=True)
    df[col] = df[col].replace("�ll", '\'ll', regex=True)
    df[col] = df[col].replace("�d", '\'d', regex=True)
    df[col] = df[col].replace(" � ", ' ', regex=True)
    df[col] = df[col].replace("\"� ", '', regex=True)
    df[col] = df[col].replace("�", '', regex=True)
    df[col] = df[col].replace("è", 'e', regex=True)
    df[col] = df[col].replace("é", 'e', regex=True)
    df[col] = df[col].replace('(?:\s+\*)+', '', regex=True)
    df[col] = df[col].replace('(?:\s+\.)+', '.', regex=True)
    df[col] = df[col].replace('\.{2,}', '.', regex=True)
    df[col] = df[col].replace('\s{2,}', ' ', regex=True)
    df[col] = df[col].replace('\-{2,}', ' ', regex=True)
    df[col] = df[col].replace('\&{2,}', ' ', regex=True)
    df[col] = df[col].replace('\.\?', '?', regex=True)
    df[col] = df[col].replace('^\s+', '', regex=True)

  
  return df

In [27]:
df_mov = pd.read_csv('gdrive/My Drive/Colab Notebooks/CRF/data/movie_quotes_data.csv')
df_mov

,sent_id,sentence,prev,match,next
0,1,Because she'll scare them away. Nine-thirty right? I'm early. I'm driving. Who knocked up your s...,Because she'll scare them away. Nine-thirty right? I'm early. I'm driving.,Who knocked up your sister?,"Nice to see you. Martini bar to the right, shots in the kitchen. Lookin' fresh tonight, Pussy-K..."
1,2,"Oh, really? You know, these guys are no Bikini Kill or The Raincoats, but they're right up there...","Oh, really? You know, these guys are no Bikini Kill or The Raincoats, but they're right up there...",I watched you out there I've never seen you look like that,Come to that party with me. What? You forgot to pay!
2,3,"Why can't we agree on this? Because you're making decisions for me. As a parent, that's my right...","Why can't we agree on this? Because you're making decisions for me. As a parent, that's my right...",You're eighteen. You don't know what you want. You won't know until you're forty-five and you ...,I want to go to an East Coast school! I want you to trust me to make my own choices. I want -- ...
3,4,"Sure, I do Why? Because then I'd have to start taking out girls who like me. Like you could find...","Sure, I do Why? Because then I'd have to start taking out girls who like me. Like you could find...",See that? Who needs affection when I've got blind hatred?,Just let me sit down. How's that? Jesus. You're like a weeble
4,5,"Well, no... Then that's all you had to say. But You always been this selfish? Just because you'r...","Well, no... Then that's all you had to say. But You always been this selfish?","Just because you're beautiful, doesn't mean you can treat people like they don't matter.","And I'm back in the saddle. Kat, my lady, you sway to the rhythm of my heart. Yippe kai-aye, bra..."
...,...,...,...,...,...
6277,6278,I'm still workin' on my GED. How long you think Brainiac's gonna' wanna' hang with that? She wor...,I'm still workin' on my GED. How long you think Brainiac's gonna' wanna' hang with that? She wor...,You're gonna' have a great time with her tonight. Just be yourself. ...Mostly.,"I caught him with my best friend. Former. Such a cliche'. Sorry. Winthrop Hawkins. ""Hawk."" From ..."
6278,6279,"Why're you so resistant? If the truth is, you want to be a guy, great, but if not... Don't even ...","Why're you so resistant? If the truth is, you want to be a guy, great, but if not... Don't even ...","That's the thing about the truth. It'll set you free, but first it can really piss you off.",This is nuts! I'm supposed to be a guy! Then what's the problem? You are one. You're not gonna' ...
6279,6280,Ride to Lord Chelmsford. Ride toward Ulundi. Tell him the battle he longs for has started and he...,Ride to Lord Chelmsford. Ride toward Ulundi. Tell him the battle he longs for has started and he...,"Oh no. Come all this bloody way to get shot by a bullet from Birmingham. Shoot straight, you b...",Fire one. Fire two. Reinforcement only. And ride to Stuart Smith. Let his guns cover Durnford fo...
6280,6281,"Quickly, Trooper. Well done, Vereker. Now goodbye, lad. Go on. Sergeand Well fought, Gentlemen. ...","Quickly, Trooper. Well done, Vereker. Now goodbye, lad. Go on. Sergeand","Well fought, Gentlemen. It's time to save the Colours. Get to Rorke '5 Drift. You must warn them...","Carry them to safety Mr. MelvilL Sir. watches, moves back into his tent."


In [34]:
df_mov = clean(df_mov)
df_mov['sentence'] = df_mov['prev'] + ' ' + df_mov['match'] + ' ' + df_mov['next']
df_mov

,sent_id,sentence,prev,match,next
0,1,Because she'll scare them away. Nine-thirty right? I'm early. I'm driving. Who knocked up your s...,Because she'll scare them away. Nine-thirty right? I'm early. I'm driving.,Who knocked up your sister?,"Nice to see you. Martini bar to the right, shots in the kitchen. Lookin' fresh tonight, Pussy-Ka..."
1,2,"Oh, really? You know, these guys are no Bikini Kill or The Raincoats, but they're right up there...","Oh, really? You know, these guys are no Bikini Kill or The Raincoats, but they're right up there...",I watched you out there I've never seen you look like that,Come to that party with me. What? You forgot to pay!
2,3,"Why can't we agree on this? Because you're making decisions for me. As a parent, that's my right...","Why can't we agree on this? Because you're making decisions for me. As a parent, that's my right...",You're eighteen. You don't know what you want. You won't know until you're forty-five and you do...,"I want to go to an East Coast school! I want you to trust me to make my own choices. I want, Chr..."
3,4,"Sure, I do Why? Because then I'd have to start taking out girls who like me. Like you could find...","Sure, I do Why? Because then I'd have to start taking out girls who like me. Like you could find...",See that? Who needs affection when I've got blind hatred?,Just let me sit down. How's that? Jesus. You're like a weeble
4,5,"Well, no. Then that's all you had to say. But You always been this selfish? Just because you're ...","Well, no. Then that's all you had to say. But You always been this selfish?","Just because you're beautiful, doesn't mean you can treat people like they don't matter.","And I'm back in the saddle. Kat, my lady, you sway to the rhythm of my heart. Yippe kai-aye, bra..."
...,...,...,...,...,...
6277,6278,I'm still workin' on my GED. How long you think Brainiac's gonna' wanna' hang with that? She wor...,I'm still workin' on my GED. How long you think Brainiac's gonna' wanna' hang with that? She wor...,You're gonna' have a great time with her tonight. Just be yourself. Mostly.,I caught him with my best friend. Former. Such a cliche'. Sorry. Winthrop Hawkins. 'Hawk.' From ...
6278,6279,"Why're you so resistant? If the truth is, you want to be a guy, great, but if not. Don't even sa...","Why're you so resistant? If the truth is, you want to be a guy, great, but if not. Don't even sa...","That's the thing about the truth. It'll set you free, but first it can really piss you off.",This is nuts! I'm supposed to be a guy! Then what's the problem? You are one. You're not gonna' ...
6279,6280,Ride to Lord Chelmsford. Ride toward Ulundi. Tell him the battle he longs for has started and he...,Ride to Lord Chelmsford. Ride toward Ulundi. Tell him the battle he longs for has started and he...,"Oh no. Come all this bloody way to get shot by a bullet from Birmingham. Shoot straight, you bas...",Fire one. Fire two. Reinforcement only. And ride to Stuart Smith. Let his guns cover Durnford fo...
6280,6281,"Quickly, Trooper. Well done, Vereker. Now goodbye, lad. Go on. Sergeand Well fought, Gentlemen. ...","Quickly, Trooper. Well done, Vereker. Now goodbye, lad. Go on. Sergeand","Well fought, Gentlemen. It's time to save the Colours. Get to Rorke '5 Drift. You must warn them...","Carry them to safety Mr. MelvilL Sir. watches, moves back into his tent."


In [35]:
df_mov.to_csv("gdrive/My Drive/Colab Notebooks/CRF/data/qtask_all_mov.csv",index=False)